In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import io
from pprint import pprint

from google.cloud import vision
from google.cloud.vision import types

from context import backend
from backend.apps.detections import fix_ros_path  # needs to be before cv2 import
import cv2

In [ ]:
FILE_PATH = '/home/matt/polybox/Fotos/new.jpg'
def read_img_bin(path):
    with io.open(path, 'rb') as image_file:
        return image_file.read()
    
def read_img_cv(path):
    return cv2.imread(path)
    
img_cv = read_img_cv(FILE_PATH)
img_bin = read_img_bin(FILE_PATH)
img = image_resize(img_cv, height=700)

In [ ]:
from backend.apps.detections.gcv_api import VisionClient 
vision_client = VisionClient()

In [ ]:
objects = vision_client.localize_objects(img_bin)
#objects_dict_coords_to_pic(objects, img.shape)

In [ ]:
objects

In [ ]:
pprint(objects)

In [ ]:
from backend.apps.detections.cv_helpers import image_resize
from backend.apps.detections.gcv_api import bb_norm_to_corner_pixels, objects_dict_coords_to_pic

In [ ]:
"""Plotting"""

font                   = cv2.FONT_HERSHEY_DUPLEX
fontScale              = 0.5
fontColor              = (255, 255, 255)
lineType               = 1

img_cv = read_img_cv(FILE_PATH)
img_bin = read_img_bin(FILE_PATH)
img = image_resize(img_cv, height=700)

for pred in objects:
    bbox_pix = pred['bb_vertices']  
    top_left_x, top_left_y = bbox_pix[0]['x'], bbox_pix[0]['y']
    bottom_right_x, bottom_right_y = bbox_pix[2]['x'], bbox_pix[2]['y']
    img = cv2.rectangle(img, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0,0,255), 2)
    img = cv2.putText(img,f'{pred["obj_name"]}, acc: {round(pred["confidence"], 2)}', 
        (top_left_x, top_left_y-5), 
        font, 
        fontScale,
        fontColor,
        lineType)
    
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()